given a list of user-track find the number of total replays and total songs listened per user 

In [1]:
import pyspark
from random import randint, random, choice
import matplotlib.pyplot as plt

sc = pyspark.SparkContext('local[*]')

In [2]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(idGenerator(),prefix="user",min=1,max=100,keyName="userId")
dataset.addGenerator(idGenerator(),prefix="track",min=1,max=5000,keyName="trackId")

rdd = sc.parallelize(dataset.generateDataset(250000)).persist()
rdd.take(5)

[{'userId': 'user_0084', 'trackId': 'track_04328'},
 {'userId': 'user_0055', 'trackId': 'track_03619'},
 {'userId': 'user_0062', 'trackId': 'track_00205'},
 {'userId': 'user_0044', 'trackId': 'track_03379'},
 {'userId': 'user_0058', 'trackId': 'track_02566'}]

In [10]:
# pattern count occurrences da joinovertimedataset
# count occurrences of each user-track
rdd.map(lambda x: ((x.get("userId"),x.get("trackId")),1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[(('user_0033', 'track_02469'), 6),
 (('user_0094', 'track_04085'), 6),
 (('user_0086', 'track_00801'), 6),
 (('user_0015', 'track_02924'), 6),
 (('user_0096', 'track_04441'), 6)]

In [12]:
# non serve il distinct perche' STO CONTANDO LE OCCURRENCES DI (USER,TRACK), NON SOLO USER:
rdd.map(lambda x: (x.get("userId"),x.get("trackId"))).distinct().map(lambda x: ((x[0],x[1]),1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[(('user_0033', 'track_01029'), 1),
 (('user_0020', 'track_02254'), 1),
 (('user_0015', 'track_04030'), 1),
 (('user_0043', 'track_04407'), 1),
 (('user_0023', 'track_01637'), 1)]

In [4]:
# how many replays per user
rdd.map(lambda x: (x.get("userId"), 1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[('user_0046', 2619),
 ('user_0061', 2607),
 ('user_0059', 2601),
 ('user_0017', 2599),
 ('user_0095', 2596)]

In [8]:
# how many songs per user COUNTO CCURRENCES DI SOLO USER QUINDI HA BISOGNO DI DISTINCT
rdd.map(lambda x: (x.get("userId"), x.get("trackId"))).distinct().map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[('user_0017', 2049),
 ('user_0059', 2046),
 ('user_0061', 2046),
 ('user_0027', 2036),
 ('user_0095', 2031)]